<a href="https://colab.research.google.com/github/eldadHaber/EOAS555/blob/main/diffusionReaction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from scipy import sparse
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [ ]:
def getGradientMatrix(nx, ny):

    e, e_neg = -np.ones(nx), np.ones(nx)
    #e_neg[-1] = 0
    Dx1D = sparse.spdiags([e, e_neg], [0, 1], nx-1, nx)
    e, e_neg = -np.ones(ny), np.ones(ny)
    #e_neg[-1] = 0
    Dy1D = sparse.spdiags([e, e_neg], [0, 1], ny-1, ny)
    Dx = sparse.kron(sparse.eye(ny), Dx1D)
    Dy = sparse.kron(Dy1D, sparse.eye(nx))
    D = sparse.vstack([Dx, Dy])
    return D, Dx, Dy

In [ ]:
nx = 128; ny = 129
G, Dx, Dy = getGradientMatrix(nx, ny)


In [ ]:
def getWeightedLap(Sigma):

    Sigma = Sigma.numpy()
    nx, ny = Sigma.shape
    SigmaX = (Sigma[1:,:] + Sigma[:-1,:])/2
    SigmaY = (Sigma[:,1:] + Sigma[:,:-1])/2
    SigmaF = np.hstack((SigmaX.reshape((nx-1)*ny),SigmaY.reshape((ny-1)*nx)))
    D, Dx, Dy = getGradientMatrix(nx, ny)
    SigmaMat = sparse.spdiags(SigmaF, 0, (nx-1)*ny + (ny-1)*nx, (nx-1)*ny + (ny-1)*nx )

    A = D.T @ SigmaMat @ D

    # Convert to pytorch
    As = sparse.coo_matrix(A, dtype=sparse.coo_matrix)

    values = As.data
    indices = np.vstack((As.row, As.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values.tolist())
    sh = As.shape

    Atrch = torch.sparse.FloatTensor(i, v, torch.Size(sh))

    return Atrch



In [ ]:
Sigma = torch.rand(256,257)
A = getWeightedLap(Sigma)

#plt.imshow(A.to_dense())
print(A.shape)

torch.Size([65792, 65792])


In [ ]:
def pcgsol(x, A, b, gamma, iter=100, tol=1e-3):
    # Solve 
    # (1/gamma*A + I)*x = b

    r = b  - (A @ x) / gamma - x
    p = r.clone()

    normr0 = r.norm()

    for i in range(iter):

        Ap = (A @ p) / gamma + p
        alpha = torch.sum(r ** 2) / torch.sum(p * Ap)
        x = x + alpha * p
        rnew = b  - (A @ x) / gamma - x # can be replaced with r - alpha*Ap
        beta = torch.sum(rnew ** 2) / torch.sum(r ** 2)
        r = rnew.clone()
        p = r + beta * p

        if r.norm() / normr0 < tol:
            return x

        print('iter = %2d   residual = %3.2e'%(i,r.norm()/normr0));
    return x

In [ ]:
b = torch.randn(65792,1)
x = torch.randn(65792,1)
x = pcgsol(x, A, b, 1e1,iter=100, tol=1e-3)



iter =  0   residual = 9.67e-02
iter =  1   residual = 1.01e-02
iter =  2   residual = 1.04e-03


In [ ]:
x

tensor([[-0.0319],
        [-0.0645],
        [-0.0265],
        ...,
        [-0.0508],
        [-0.1061],
        [ 0.0223]])